# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 05**: Data pipeline with Neo4j

**Date**: October 2nd 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

# Dataset description

# Data ingestion

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6025dc1b-bde8-4d46-97d1-fb66cb1225e1;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [2]:
# Build schema
# Import your module
from pcamarillor.spark_utils import SparkUtils
from pyspark.sql.functions import when, concat, col, lit

from pcamarillor.spark_utils import SparkUtils
imdb = SparkUtils.generate_schema([("user_id1", "string"), ("user_id2", "string"), ("amount", "string")])

base_path = "/opt/spark/work-dir/data/"
df_imdb = spark.read \
                .option("header", "true") \
                .schema(imdb) \
                .csv(base_path + "imdb_edgelist.csv")



# Transformations

In [3]:
# Add the code for your transformations to create nodes and edges DataFrames HERE

from pyspark.sql.functions import col

# --- NODES ---

# User Nodes
person1 = df_imdb.select(
    col("user_id1").alias("id"),
).dropDuplicates(["id"])
person1.show()

# Product Nodes
person2 = df_imdb.select(
    col("user_id2").alias("id")
).dropDuplicates(["id"])
person2.show()


# --- EDGES ---

# Purchases: relationships between User -> Product
coincidences_edges = df_imdb.select(
    col("user_id1").alias("src"),   # source node (User)
    col("user_id2").alias("dst"),   # destination node (Product)

)
coincidences_edges.show()

+---------+
|       id|
+---------+
|nm0004709|
|nm0000354|
|nm0350453|
|nm0911320|
|nm2103508|
|nm1012343|
|nm0694893|
|nm0068551|
|nm2162912|
|nm0000362|
|nm2244829|
|nm2398362|
|nm0744776|
|nm4270477|
|nm0000198|
|nm1172478|
|nm6239893|
|nm4667815|
|nm3603254|
|nm5465692|
+---------+
only showing top 20 rows


+----------+
|        id|
+----------+
| nm0350453|
| nm0000198|
| nm0694893|
| nm0000354|
| nm0004709|
| nm0068551|
| nm3236159|
|nm12103226|
| nm3478396|
| nm0000362|
| nm1172478|
| nm0005258|
| nm4802218|
| nm0911320|
| nm2103508|
| nm0191412|
| nm1480555|
| nm0201638|
| nm4480988|
| nm0005349|
+----------+
only showing top 20 rows
+---------+---------+
|      src|      dst|
+---------+---------+
|nm3662667|nm1568415|
|nm3662667|nm3421365|
|nm3662667|nm3661490|
|nm1568415|nm3421365|
|nm1568415|nm3661490|
|nm1568415|nm0219967|
|nm1568415|nm1069034|
|nm1568415|nm0348004|
|nm3421365|nm3661490|
|nm0344435|nm0004821|
|nm0344435|nm0262635|
|nm0344435|nm0004695|
|nm0344435|nm2301950|
|nm0344435|nm0001057|
|nm0344435|nm0000347|
|nm0344435|nm0828980|
|nm0344435|nm0006969|
|nm0344435|nm0000982|
|nm0344435|nm0004988|
|nm0344435|nm0001290|
+---------+---------+
only showing top 20 rows


# Writing Data in Neo4j

In [ ]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"

person1.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":person") \
  .option("node.keys", "id") \
  .save()

print(f"{person1.count()} users wrote in Neo4j")


person2.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":person") \
  .option("node.keys", "id") \
  .save()

print(f"{person2.count()} product nodes wrote in Neo4j")

coincidences_edges.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "acted") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", ":person") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "src:id") \
  .option("relationship.target.labels", ":person") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "dst:id") \
  .save()

print(f"{coincidences_edges.count()} coincidences edges wrote in Neo4j")

23814 users wrote in Neo4j


[Stage 18:>                                                         (0 + 1) / 1]

# Read and Query Graphs with PySpark

In [ ]:
# Add the code to read a data frame from Neo4J and run a simple query to verify 
df_relations = spark.read \
    .format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("query",
            """
            MATCH (u:person)-[r:acted]->(p:person)
            WHERE r.amount > 1000
            RETURN u.id, p.id, r.amount
            """) \
    .load()

df_relations.show()

In [1]:
sc.stop()

NameError: name 'sc' is not defined